In [23]:
import json
import glob

import cv2

from PIL import Image

import numpy as np

# 点のペアを辞書から作成する関数
def dict_to_points(json_path):
    json_open = open(json_path)
    json_load = json.load(json_open)
    
    x_list = []
    y_list = []

    for i in range(len(json_load[0]["cgPoints"])):
        x = int(json_load[0]["cgPoints"][i]["x"])
        y = int(json_load[0]["cgPoints"][i]["y"])
        x_list.append(x)
        y_list.append(y)

    x_arr = np.array(x_list)
    y_arr = np.array(y_list)

    points = np.stack([x_arr, y_arr], axis=1)
    
    return points

# 点のペアから塗りつぶし画像を作り出す関数
def points_to_mask(points, size, save_path):
    # 真っ黒の画像を用意
    img = np.zeros((size, size), np.uint8)
    # 点の集合をarrayにする
    points = dict_to_points(json_path)

    # 塗りつぶす
    cv2.fillConvexPoly(img, points, (255, 255, 0))

    cv2.imwrite(save_path + "/mask.png", img)
    

# 入力された画像が真っ黒かどうか調べる関数
def check_black(img_path):
    img = Image.open(img_path)
    arr = np.array(img)
    if arr.max() == 0:
        flag = 1
    else:
        flag = 0
    return flag

In [24]:
slices = glob.glob("dataset/364240/sag/1.2.392.200036.9142.10002202.1020330992.2.20190905190738.28273/*")

In [25]:
#スライス枚数だけ繰り返す
for slice_ in slices: 
    img_path = slice_ + "/DicomImage.jpg"
    json_path = slice_ + "/AnnotationFile.Json"
    save_path = slice_
    #もしDicomImage.jpgが真っ黒じゃなかったら
    makkuro = check_black(img_path)
    if makkuro != True:
        points = dict_to_points(json_path)
        points_to_mask(points=points, size=640, save_path=save_path)